# Method

### 动态方法 Object#send

> send 可以调用私有方法。

```ruby
Object#send(symbol [,args])
Object#send(string [,args])

class Klass
  def hello(*args)
    "Hello " + args.join(' ')
  end
end
k = Klass.new
k.send :hello, "gentle", "readers"   #=> "Hello gentle readers"
```

In [69]:
Object.new.methods.grep(/send/)


[:public_send, :send, :__send__]

In [72]:
class Klass
  def hello(*args)
    p "Hello " + args.join(' ')
  end
end
k = Klass.new
k.hello("world!")
k.send :hello, "gentle", "readers"   #=> "Hello gentle readers"

"Hello world!"
"Hello gentle readers"


"Hello gentle readers"

In [15]:
# 来自Camping的例子


#START:configuration
# Load configuration if any
if conf.rc and File.exists?( conf.rc )
  YAML.load_file(conf.rc).each do |k,v|
    conf.send("#{k}=", v)
  end 
end
#END:configuration

TypeError: no implicit conversion of true into String

## DRY

In [4]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
    
  def mouse
    info = @data_source.get_mouse_info(@id)
    price = @data_source.get_mouse_price(@id)
    result = "Mouse: #{info} ($#{price})"
    return "* #{result}" if price >= 100
    result
  end
    
  def cpu
   info = @data_source.get_cpu_info(@id)
   price = @data_source.get_cpu_price(@id)
   result = "Cpu: #{info} ($#{price})"
   return "* #{result}" if price >= 100
   result
  end
    
  def keyboard
    info = @data_source.get_keyboard_info(@id)
    price = @data_source.get_keyboard_price(@id)
    result = "Keyboard: #{info} ($#{price})"
    return "* #{result}" if price >= 100
    result
  end
# ...
end

# 这段代码有什么问题？

:keyboard

## 动态定义方法 Module#define_method

In [77]:
class MyClass
  define_method :my_method do |my_arg|
    my_arg * 3
  end

end

obj = MyClass.new
obj.my_method(2)  # => 6
obj.send(:my_method, 3) # => 9



9

In [74]:
10.times do |i, index|
    p i
end

0
1
2
3
4
5
6
7
8
9


10

## 重构尝试一： 动态派发

使用`send`

In [22]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end

  def mouse
    component :mouse
  end

  def cpu
    component :cpu
  end

  def keyboard
    component :keyboard
  end

  def component(name)
    info = @data_source.send "get_#{name}_info", @id
    price = @data_source.send "get_#{name}_price", @id
    result = "#{name.to_s.capitalize}: #{info} ($#{price})"
    return "* #{result}" if price >= 100
    result
  end
end


:component

## 重构尝试二： 创建方法

In [78]:
class Computer
    
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
 

  def self.define_component(name)
    p "exec..."
    define_method(name) do
      info = @data_source.send "get_#{name}_info", @id
      price = @data_source.send "get_#{name}_price", @id
      result = "#{name.to_s.capitalize}: #{info} ($#{price})"
      return "* #{result}" if price >= 100
      result
    end
  end
  
  define_component :mouse
  define_component :cpu
  define_component :keyboard
end


"exec..."
"exec..."
"exec..."


:keyboard

## 重构尝试三：用内省方式缩减代码

In [80]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
    data_source.methods.grep(/^get_(.*)_info$/) { Computer.define_component $1 }
  end  

  def self.define_component(name)
    define_method(name) do
      info = @data_source.send "get_#{name}_info", @id
      price = @data_source.send "get_#{name}_price", @id
      result = "#{name.capitalize}: #{info} ($#{price})"
      return "* #{result}" if price >= 100
      result
    end
  end
end


:define_component

# [method_missing](https://ruby-doc.org/core-2.6.3/BasicObject.html)

```ruby
class Roman
  def roman_to_int(str)
    # ...
  end
  def method_missing(methId)
    str = methId.id2name
    roman_to_int(str)
  end
end

r = Roman.new
r.iv      #=> 4
```


In [86]:
# sample
class Lawyer
  def method_missing(method, *args)
    puts "You called: #{method}(#{args.join(', ')})"
    puts "(You also passed it a block)" if block_given?
  end
end

Lawyer.new.hello(:world){}



You called: hello(world)
(You also passed it a block)


# 幽灵方法

In [49]:

require 'ruport'
table = Ruport::Data::Table.new :column_names => ["country", "wine"],
                                :data => [["France", "Bordeaux"],
                                          ["Italy", "Chianti"],
                                          ["France", "Chablis"]]
puts table.to_text

found = table.rows_with_country("France")
found.each do |row|
  puts row.to_csv
end


LoadError: cannot load such file -- ruport

In [50]:
 # Provides a shortcut for the <tt>as()</tt> method by converting a call to
    # <tt>as(:format_name)</tt> into a call to <tt>to_format_name</tt>
    #
    # Also converts a call to <tt>rows_with_columnname</tt> to a call to
    # <tt>rows_with(:columnname => args[0])</tt>.
    #
  def method_missing(id,*args,&block)
   return as($1.to_sym,*args,&block) if id.to_s =~ /^to_(.*)/ 
   return rows_with($1.to_sym => args[0]) if id.to_s =~ /^rows_with_(.*)/
   super
  end


:method_missing

## 补充知识点



In [53]:
"adb".to_sym
"bcd".intern

:bcd

In [89]:
:abc.to_s
:abc.id2name

"abc"

## 来自Ruby核心库的例子

In [92]:
require 'ostruct'
icecream = OpenStruct.new
icecream.flavor=("strawberry")
icecream.flavor # => "strawberry"

"strawberry"

## 简化版的Openstruct

In [93]:
class MyOpenStruct
  def initialize
    @attributes = {}
  end
    
  def method_missing(name, *args)
    attribute = name.to_s
    if attribute =~ /=$/
      @attributes[attribute.chop] = args[0]
    else
      @attributes[attribute]
    end
  end
end
icecream = MyOpenStruct.new
icecream.flavor = "vanilla"
icecream.flavor

"vanilla"

## 动态代理

一个捕获幽灵方法调用并把他们转发给另外一个对象的对象，有时在转发前后会包装一些自己的逻辑

In [63]:
def method_missing(method_id, *params)
  request(method_id.id2name.gsub(/_/, '.'), params[0])
end

:method_missing

In [66]:
require 'delegate'
class Assistant
  def initialize(name)
    @name = name
  end
    
  def read_email
    "(#{@name}) It's mostly spam."
  end
  
  def check_schedule
    "(#{@name}) You have a meeting today."
  end
end

class Manager < DelegateClass(Assistant)
  def initialize(assistant)
    super(assistant)
  end
  
  def attend_meeting
    "Please hold my calls."
  end
end

frank = Assistant.new("Frank")
anne = Manager.new(frank)
anne.attend_meeting     # => "Please hold my calls."
anne.read_email         # => "(Frank) It's mostly spam."
anne.check_schedule     # => "(Frank) You have a meeting today."

"(Frank) You have a meeting today."

`DelegateClass()`是一种拟态方法， 创建并返回一个新的`Class`

这个类会定义一个`method_missing`方法，并把对它发生的调用转发到被封装的对象上。

## 终极重构

In [71]:
class DS
  def initialize # connect to data source...
  end

  def get_mouse_info(workstation_id) # ...
    "Dual Optical"
  end
  
  def get_mouse_price(workstation_id) # ...
    40
  end
  
  def get_keyboard_info(workstation_id) # ...
    "Standard UK"
  end
  
  def get_keyboard_price(workstation_id) # ...
    20
  end

  def get_cpu_info(workstation_id) # ...
      "2.16 Ghz"
    end

  def get_cpu_price(workstation_id) # ...
    220
  end

  def get_display_info(workstation_id) # ...
    "LED 1280x1024"
  end

  def get_display_price(workstation_id) # ...
  # ...and so on
    150
  end
end

:get_display_price

In [75]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
    
  def method_missing(name, *args)
    super unless @data_source.respond_to?("get_#{name}_info")
    info = @data_source.send("get_#{name}_info", args[0])
    price = @data_source.send("get_#{name}_price", args[0])
    brand = @data_source.send("get_#{name}_brand", args[0])
    result = "#{name.to_s.capitalize}: #{info} ($#{price})"
    return "* #{result}" if price >= 100
    result
  end
end

cmp = Computer.new(0, DS.new)
cmp.cpu
# cmp.respond_to?(:mous)       # => false

"* Cpu: 2.16 Ghz ($220)"

# method_missing in Rails

In [77]:
 module ClassSpecificRelation # :nodoc:
      extend ActiveSupport::Concern

      module ClassMethods # :nodoc:
        def name
          superclass.name
        end
      end

      private
        def method_missing(method, *args, &block)
          if @klass.respond_to?(method)
            @klass.generate_relation_method(method)
            scoping { @klass.public_send(method, *args, &block) }
          else
            super
          end
        end
        ruby2_keywords(:method_missing)
end

NameError: uninitialized constant #<Class:0x00007fde84250368>::ClassSpecificRelation::ActiveSupport

# Module#const_missing

In [94]:
def Object.const_missing(name)
  name.to_s.downcase.gsub(/_/, ' ')
end

MY_CONSTANT

"my constant"

## 当方法冲突时

In [99]:
class Foo
end
Foo.new.display

#<#<Class:0x00007fc075af2ef8>::Foo:0x00007fc07528b4e0>

In [83]:
Object.instance_methods.grep /^d/

[:dup, :display, :define_singleton_method]

In [103]:
class Foo
  def display;end
    
  remove_method :display
end
Foo.new.display

#<#<Class:0x00007fc075af2ef8>::Foo:0x00007fc075b80690>

In [85]:
require 'pry-doc'

true

## 性能

In [100]:
# 在此执行会有问题

class String
  def method_missing(method, *args)
    method == :ghost_reverse ? reverse : super
  end
end

"abc".ghost_reverse

require 'benchmark'

Benchmark.bm do |b|
  b.report 'Normal method' do
    1000000.times { "abc".reverse }
  end
    
  b.report 'Ghost method ' do
   1000000.times { "abc".ghost_reverse }
  end
end

NoMethodError: undefined method `ghost_reverse' for "abc":String

### 白板类

> 1.9 以后，可以使用BasicObject


In [ ]:
######################################################################
# BlankSlate provides an abstract base class with no predefined
# methods (except for <tt>\_\_send__</tt> and <tt>\_\_id__</tt>).
# BlankSlate is useful as a base class when writing classes that
# depend upon <tt>method_missing</tt> (e.g. dynamic proxies).
#
class BlankSlate
  # Hide the method named +name+ in the BlankSlate class.  Don't
  # hide +instance_eval+ or any method beginning with "__".
  def self.hide(name)
    if instance_methods.include?(name.to_s) and
      name !~ /^(__|instance_eval)/
      @hidden_methods ||= {}
      @hidden_methods[name.to_sym] = instance_method(name)
      undef_method name
    end
  end

  class << self
    
    def find_hidden_method(name)
      @hidden_methods ||= {}
      @hidden_methods[name] || superclass.find_hidden_method(name)
    end

    # Redefine a previously hidden method so that it may be called on a blank
    # slate object.
    def reveal(name)
      bound_method = nil
      unbound_method = find_hidden_method(name)
      fail "Don't know how to reveal method '#{name}'" unless unbound_method
      define_method(name) do |*args|
        bound_method ||= unbound_method.bind(self)
        bound_method.call(*args)
      end
    end
  end
  
  instance_methods.each { |m| hide(m) }
  # ...
end

## Module#undef_method

> 删除自身以及继承来的方法，删除的最彻底

## Module#remove_method

> 仅删除自身

In [111]:
class Parent
  def hello
    puts "In parent"
  end
end
   
class Child < Parent
  def hello
    puts "In child"
  end
end

c = Child.new
# c.hello

# p Child.instance_methods(false)
p Parent.instance_methods(false)

[:hello]


[:hello]

In [1]:
class Child
  remove_method :hello  # remove from child, still in parent
end

p Parent.instance_methods(false)
# p Child.instance_methods(false)

NameError: method `hello' not defined in #<Class:0x00007fa2db2d62e0>::Child

In [4]:
class Parent1
  def hello
    puts "In parent"
  end
end
   
class Child1 < Parent1
  def hello
    puts "In child"
  end
end

class Child1
  undef_method :hello   # prevent any calls to 'hello'
end

c = Child1.new
c.hello

NoMethodError: undefined method `hello' for #<#<Class:0x00007fa2db2d62e0>::Child1:0x00007fa2db26c8b8>

In [ ]:
static VALUE
rb_mod_undef_method(int argc, VALUE *argv, VALUE mod)
{
    int i;
    for (i = 0; i < argc; i++) {
        VALUE v = argv[i];
        ID id = rb_check_id(&v);
        if (!id) {
          rb_method_name_error(mod, v);
        }
        rb_undef(mod, id);
    }
    return mod;
}

# 回顾yield

In [7]:
def a_method a,b
  a + yield(a,b)
  a + yield(a,b)
end

a_method(1, 2)

LocalJumpError: no block given (yield)

## Kernel#block_given?

In [8]:
def a_method a,b
  a + yield(a,b) if block_given?
  a + b
end

a_method(1, 2)

3

# using 关键字

`using` 希望`conn` 有个`dispose` 的方法，保证资源得到释放

```c#
RemoteConnection conn = new RemoteConnection("some_remote_server");
using (conn)
{
    conn.readSomeData();
    doSomeMoreStuff();
}

```

In [25]:
module Kernel
  def using(resource)
    begin
      yield
    ensure
      resource.dispose
    end
   end
end

:using

In [26]:
# 测试程序


require 'test/unit'

class TestUsing < Test::Unit::TestCase
  class Resource
    def dispose
      @disposed = true
    end

    def disposed?
      @disposed
    end
  end

  def test_disposes_of_resources
    r = Resource.new
    using(r) {}
    assert r.disposed?
  end
  
  def test_disposes_of_resources_in_case_of_exception
    r = Resource.new
    assert_raises(Exception) {
      using(r) {
        raise Exception
      }
    }
    assert r.disposed?
  end
end


:test_disposes_of_resources_in_case_of_exception

# Lamba和Proc：把函数当成一等公民

In [11]:
def msg_func
  str = "The quick brown fox"
  lambda do |animal|
    p "#{str} jumps over the lazay #{animal}"
  end
end

func = msg_func
func.call('dog')

"The quick brown fox jumps over the lazay dog"


"The quick brown fox jumps over the lazay dog"

<img src="assets/proc.png" alt="Drawing" style="width: 600px; height: 600px"/>

### Ruby 会避免栈帧的多次复制

In [19]:
## 所以复用了 i 的值，并没有在 lambda 内部保持 i 的值
i = 0

inc_func = -> do
  p "Inc from #{i} to #{i+1}"
  i += 1
end

dec_func = -> do
  i -= 1
  p "Dec from #{i+1} to #{i}"
end

inc_func.call
dec_func[]
inc_func.===
inc_func.===
dec_func[]

"Inc from 0 to 1"
"Dec from 1 to 0"
"Inc from 0 to 1"
"Inc from 1 to 2"
"Dec from 2 to 1"


"Dec from 2 to 1"

# 闭包（Closure）

<img src="assets/code.png" alt="Drawing" style="width: 600px; height: 600px"/>

### 可运行代码： 代码 + 绑定

In [12]:
def my_method
  x = "Not Binding"
  yield("cruel")
end

x = "Binding"
my_method {|y| "#{x}, #{y} world" } # => "Hello, cruel world"


"Binding, cruel world"

# 作用域Scope

**Ruby中的作用域是截然分开的！！！**

In [21]:
# Kernel#local_varialbles 

v1 = 1 

class MyClass
  # p v1  # 不可见 NameError: undefined local variable or method `v1' for #<Class:0x00007fc075af2ef8>::MyClass
  v2 = 2                
  local_variables    # => [:v2]
  def my_method
    # p v2
    v3 = 3
    local_variables
  end
  local_variables    # => [:v2]
end

obj = MyClass.new
obj.my_method        # => [:v3]
local_variables.grep /^[^_]/      # => [:v1, :obj]

[:obj, :v1, :x, :func, :c]

### 作用域门

* 类定义
* 模块定义
* 方法

<img src="assets/scope.png" alt="Drawing" style="width: 600px; height: 400px"/>

## 全局变量和顶尖实例变量

In [23]:
def a_scope
  $var = "some value"
end

def another_scope
  $var
end

a_scope
another_scope # => "some value"

"some value"

In [26]:

@var = "The top-level @var"

def my_method
  @var
end

my_method # => "The top-level @var"


"The top-level @var"

In [39]:
self.class

Object

In [29]:
class MyClass
  def my_method
    @var = "This is not the top-level @var!"
  end
end

MyClass.new.instance_variables
local_variables.grep  /^[^_]/

[:obj, :v1, :x, :func, :c]

## 扁平化作用域

场景：希望在方法之间共享一个变量，但是又不希望其他方法访问它

In [49]:
my_var = "Success"
class MyClass
  # We want to print my_var here...
    
  def my_method
    # ..and here
  end
end


:my_method

### Class.new == class

In [33]:
my_var = "Success"

MyClass = Class.new do
  # Now we can print my_var here...
  #puts "#{my_var} in the class definition!"

  def my_method
    # ...but how can we print it here?
    puts "#{my_var} in the method definition!"
  end
end
MyClass.new.my_method

(irb):2: warning: already initialized constant Object::MyClass
(irb):2: warning: previous definition of MyClass was here


NameError: undefined local variable or method `my_var' for #<#<Class:0x00007fa2db1889d8>:0x00007fa2db180670>

# Module#define_method

In [34]:
my_var = "Success"

MyClass = Class.new do
  puts "#{my_var} in the class definition!"

  define_method :my_method do
    puts "#{my_var} in the method!"
  end
end

MyClass.new.my_method

Success in the class definition!


(irb):2: warning: already initialized constant Object::MyClass
(irb):2: warning: previous definition of MyClass was here


Success in the method!


# 作用域小结

### 作用域门
* class
* module
* def


### 关键技术
* Class.new
* Module.new
* define_method


# Object#instance_eval
 
> instance_eval(string [, filename [, lineno]] ) → obj

> instance_eval {|obj| block } → obj

Evaluates a string containing Ruby source code, or the given block, within the context of the receiver (obj). In order to set the context, the variable self is set to obj while the code is executing, giving the code access to obj's instance variables and private methods.
  When instance_eval is given a block, obj is also passed in as the block's only argument.
  When instance_eval is given a String, the optional second and third parameters supply a filename and starting line number that are used when reporting compilation errors.

```ruby
class KlassWithSecret
  def initialize
    @secret = 99
  end
  private
  def the_secret
    "Ssssh! The secret is #{@secret}."
  end
end
k = KlassWithSecret.new
k.instance_eval { @secret }          #=> 99
k.instance_eval { the_secret }       #=> "Ssssh! The secret is 99."
k.instance_eval {|obj| obj == self } #=> true
```


In [35]:

class MyClass
  def initialize
    @v = 1
  end
end

obj = MyClass.new
obj.instance_eval "p 'hello'"


"hello"


"hello"

In [36]:
v=2
obj.instance_eval do
  self # => #<MyClass:0x3340dc @v=1>
  @v # => 1
end

obj.instance_eval { @v = v }
obj.instance_eval { @v } # => 2

2

In [56]:
obj.instance_eval "p 'hello'"

"hello"


"hello"

# Object#instance_exec

> instance_exec(arg...) {|var...| block } → obj

Executes the given block within the context of the receiver (obj). 
In order to set the context, the variable self is set to obj while the code is executing, 
giving the code access to obj's instance variables. 
Arguments are passed as block parameters.

```ruby
class KlassWithSecret
  def initialize
    @secret = 99
  end
end
k = KlassWithSecret.new
k.instance_exec(5) {|x| @secret+x }   #=> 104
```

In [37]:
class C
  def initialize
   @x, @y = 1, 2
  end
end

C.new.instance_exec(3) {|arg| (@x + @y) * arg }  # => 9
# C.new.instance_exec("p 'hello'")

9

## instance_exec VS instance_eval

### instance_exec 允许传入参数
### instance_exec 不允许传入字符串

# 洁净室

有时创建一个对象，仅仅为了在其中执行块。这样的对象称为洁净室。



In [62]:
class CleanRoom
  def complex_calculation
    # ...
    11
  end
  
  def do_something
    # ...
  end
end

clean_room = CleanRoom.new
clean_room.instance_eval do
  if complex_calculation > 10
    do_something
  end
end


## 可调用对象

* proc
* Proc
* lambda


In [64]:
dec = lambda {|x| x - 1 }
dec.class # => Proc
dec.call(2) # => 1

1

# & 操作符

* 把这个块传递给另外一个方法
* 把这个块转为Proc


In [66]:
def math(a, b)
  yield(a, b)
end

def teach_math(a, b, &operation)
  puts "Let's do the math:"
  puts math(a, b, &operation)
end

teach_math(2, 3) {|x, y| x * y}

Let's do the math:
6


# 回顾：proc，lambda区别

* 定义方式不同
* return 不同
* 对待参数的方式不同

# 重访方法

In [68]:
cladd MyClass
  def initialize(value)
    @x = value
  end

  def my_method
    @x
  end
end

object = MyClass.new(1)
m = object.method :my_method
m.call

SyntaxError: (irb):8: syntax error, unexpected `end', expecting end-of-input